# Task 1: Fuzzy match ar_buyer and cus_name

In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
import jieba
import os
import re

In [2]:
# Define root directory
root_dir = r"D:/铁书宁/2025 HKU predoc/RA task1_20250405"

df_path = os.path.join(root_dir, "ar-sale-buyer-match_forShuning.csv")
df = pd.read_csv(df_path)

### Step 1: City match for the AR buyer and customer.

In [3]:
# Import region dataset
city_path = os.path.join(root_dir, "China-City-List-latest.csv")
city_df = pd.read_csv(city_path, encoding = 'utf-8', header = 1)

# Keep city names
city_df = city_df[['Adm1_Name_ZH','Adm2_Name_ZH']]
city_df = city_df.drop_duplicates()

# Delele the word "省" and "市"
city_df['Adm1_Name_ZH'] = city_df['Adm1_Name_ZH'].str[:-1]
city_df['Adm2_Name_ZH'] = city_df['Adm2_Name_ZH'].str[:-1]

# Generate a region list
regions = pd.concat([city_df['Adm1_Name_ZH'], city_df['Adm2_Name_ZH']])
regions = list(regions.drop_duplicates())

In [18]:
# Define a region extract function
def extract_region(name):
    for region in regions:
        if region in name:
            return region
    return None

In [ ]:
df['ar_buyer_region'] = df['ar_buyer_standard'].astype(str).apply(extract_region)
df['cus_region'] = df['cus_name_standard'].astype(str).apply(extract_region)

In [61]:
# City match for ar_buyer and cus_name, also keep the N.A.
df_citymatch = df[
    (df['ar_buyer_region'] == df['cus_region']) |
    (df['ar_buyer_region'].isna() & df['cus_region'].isna())
]

In [62]:
df_citymatch.head(10)

,wind_code,sec_name,ar_by_buyer_name,ar_buyer_standard,ar_buyer_subid,cus_name,cus_name_standard,cus_name_subid,ar_buyer_region,cus_region
77,000004.SZ,国华网安,内蒙古公安厅,内蒙古公安厅,6,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None
107,000004.SZ,国华网安,周文亮,周文亮,8,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None
121,000004.SZ,国华网安,四川科伦医药贸易有限公司,四川科伦医药贸易有限公司,9,四川源亨众生医药有限公司,四川源亨众生医药有限公司,2,四川,四川
136,000004.SZ,国华网安,国药控股四川医药股份有限公司,国药控股四川医药股份有限公司,10,四川源亨众生医药有限公司,四川源亨众生医药有限公司,2,四川,四川
152,000004.SZ,国华网安,宁夏耀康医药有限公司,宁夏耀康医药有限公司,11,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None
183,000004.SZ,国华网安,山东瑞康医药股份有限公司,山东瑞康医药股份有限公司,13,山东众智医药有限公司,山东众智医药有限公司,4,山东,山东
198,000004.SZ,国华网安,山东豪诺医药有限公司,山东豪诺医药有限公司,14,山东众智医药有限公司,山东众智医药有限公司,4,山东,山东
302,000004.SZ,国华网安,杜秀珍,杜秀珍,21,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None
317,000004.SZ,国华网安,"杨秀兰,丁亮","杨秀兰,丁亮",22,国药乐仁堂医药有限公司,国药乐仁堂医药有限公司,3,None,None
351,000004.SZ,国华网安,河北龙海新药经营有限公司,河北龙海新药经营有限公司,24,河北启源医药有限公司,河北启源医药有限公司,7,河北,河北


### Step 2: Use Jieba to seperate the string to words.

In [63]:
# Use jieba to seperate the words.
df_citymatch = df_citymatch.copy()
df_citymatch['ar_buyer_seg'] = df_citymatch['ar_buyer_standard'].astype(str).apply(lambda x: jieba.lcut(x))
df_citymatch['cus_name_seg'] = df_citymatch['cus_name_standard'].astype(str).apply(lambda x: jieba.lcut(x))

In [64]:
df_citymatch = df_citymatch.copy()
df_citymatch['cus_name_seg_str'] = df_citymatch['cus_name_seg'].apply(lambda x: ' '.join(x))
df_citymatch['ar_buyer_seg_str'] = df_citymatch['ar_buyer_seg'].apply(lambda x: ' '.join(x))

#### Jieba just for Chinese names, so for English names, we normalize the text by removing spaces and punctuation.

In [65]:
# Determine if it is English name
def is_english(text):
    return bool(re.match(r'^[\x00-\x7F]+$', text))

# Retain letters and numbers, remove special symbols such as spaces, punctuation, etc., and convert to lower case
def clean_english_text(text):
    return re.sub(r'[^a-zA-Z0-9]', '', text).lower()

In [69]:
df_citymatch['ar_buyer_seg_str'] = df_citymatch['ar_buyer_seg_str'].astype(str).apply(
    lambda x: clean_english_text(x) if is_english(x) else x
)

df_citymatch['cus_name_seg_str'] = df_citymatch['cus_name_seg_str'].astype(str).apply(
    lambda x: clean_english_text(x) if is_english(x) else x
)

### Step 3: Compute fuzzy match score

In [70]:
df_fuzzymatch = df_citymatch.iloc[:,:]

# Fuzzy match function
def fuzzy_match(name1, name2):
    return fuzz.token_sort_ratio(name1, name2)

# Apply fuzzy matching to your columns
df_fuzzymatch['match_score'] = df_fuzzymatch.apply(lambda row: fuzzy_match(row['cus_name_seg_str'], row['ar_buyer_seg_str']), axis=1)

In [79]:
output_path1 = os.path.join(root_dir, "df_fuzzymatch_v2.csv")
df_fuzzymatch.to_csv(output_path1, encoding = 'utf-8-sig')

#### (1) Choose 85 as a cutoff score

In [71]:
df_fuzzymatch85 = df_fuzzymatch[df_fuzzymatch['match_score'] >= 85]
df_fuzzymatch85.head(10)

,wind_code,sec_name,ar_by_buyer_name,ar_buyer_standard,ar_buyer_subid,cus_name,cus_name_standard,cus_name_subid,ar_buyer_region,cus_region,ar_buyer_seg,cus_name_seg,cus_name_seg_str,ar_buyer_seg_str,match_score
430,000004.SZ,国华网安,福建乐游网络科技有限公司,福建乐游网络科技有限公司,29,福建乐游网络科技有限公司,福建乐游网络科技有限公司,11,福建,福建,"[福建, 乐游, 网络科技, 有限公司]","[福建, 乐游, 网络科技, 有限公司]",福建 乐游 网络科技 有限公司,福建 乐游 网络科技 有限公司,100
432,000004.SZ,国华网安,福建乐游网络科技有限公司,福建乐游网络科技有限公司,29,福建摩格网络科技有限公司,福建摩格网络科技有限公司,13,福建,福建,"[福建, 乐游, 网络科技, 有限公司]","[福建, 摩格, 网络科技, 有限公司]",福建 摩格 网络科技 有限公司,福建 乐游 网络科技 有限公司,87
446,000004.SZ,国华网安,福建众邦药业有限公司,福建众邦药业有限公司,30,福建众邦药业有限公司,福建众邦药业有限公司,12,福建,福建,"[福建, 众邦, 药业, 有限公司]","[福建, 众邦, 药业, 有限公司]",福建 众邦 药业 有限公司,福建 众邦 药业 有限公司,100
460,000004.SZ,国华网安,福建摩格网络科技有限公司,福建摩格网络科技有限公司,31,福建乐游网络科技有限公司,福建乐游网络科技有限公司,11,福建,福建,"[福建, 摩格, 网络科技, 有限公司]","[福建, 乐游, 网络科技, 有限公司]",福建 乐游 网络科技 有限公司,福建 摩格 网络科技 有限公司,87
462,000004.SZ,国华网安,福建摩格网络科技有限公司,福建摩格网络科技有限公司,31,福建摩格网络科技有限公司,福建摩格网络科技有限公司,13,福建,福建,"[福建, 摩格, 网络科技, 有限公司]","[福建, 摩格, 网络科技, 有限公司]",福建 摩格 网络科技 有限公司,福建 摩格 网络科技 有限公司,100
551,000006.SZ,深振业A,天虹商场股份有限公司,天虹商场股份有限公司,2,天虹商场股份有限公司,天虹商场股份有限公司,3,None,None,"[天虹, 商场, 股份, 有限公司]","[天虹, 商场, 股份, 有限公司]",天虹 商场 股份 有限公司,天虹 商场 股份 有限公司,100
599,000006.SZ,深振业A,深圳市前海同创汇产业运营管理有限公司,深圳市前海同创汇产业运营管理有限公司,7,深圳市前海同创汇产业运营管理有限公司,深圳市前海同创汇产业运营管理有限公司,6,深圳,深圳,"[深圳市, 前海同, 创汇, 产业, 运营, 管理, 有限公司]","[深圳市, 前海同, 创汇, 产业, 运营, 管理, 有限公司]",深圳市 前海同 创汇 产业 运营 管理 有限公司,深圳市 前海同 创汇 产业 运营 管理 有限公司,100
619,000006.SZ,深振业A,深圳市南山区建筑工务署,深圳市南山区建筑工务署,9,深圳市南山区建筑工务署,深圳市南山区建筑工务署,8,深圳,深圳,"[深圳市, 南山区, 建筑, 工务, 署]","[深圳市, 南山区, 建筑, 工务, 署]",深圳市 南山区 建筑 工务 署,深圳市 南山区 建筑 工务 署,100
797,000007.SZ,全新好,MAPLE PACKING INDUSTRIAL (H.K.) LIMITED,MAPLE PACKING INDUSTRIAL (H.K.) LIMITED,5,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,1,None,None,"[MAPLE, , PACKING, , INDUSTRIAL, , (, H, .,...","[MAPLEPACKINGINDUSTRIAL, (, H, ., K, ., ), LIM...",maplepackingindustrialhklimited,maplepackingindustrialhklimited,100
814,000007.SZ,全新好,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,6,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,1,None,None,"[MAPLEPACKINGINDUSTRIAL, (, H, ., K, ., ), LIM...","[MAPLEPACKINGINDUSTRIAL, (, H, ., K, ., ), LIM...",maplepackingindustrialhklimited,maplepackingindustrialhklimited,100


In [80]:
output_path2 = os.path.join(root_dir, "df_fuzzymatch85_v2.csv")
df_fuzzymatch85.to_csv(output_path2, encoding = 'utf-8-sig')

#### (1) Choose 90 as a cutoff score

In [74]:
df_fuzzymatch90 = df_fuzzymatch[df_fuzzymatch['match_score'] >= 90]
df_fuzzymatch90.head(10)

,wind_code,sec_name,ar_by_buyer_name,ar_buyer_standard,ar_buyer_subid,cus_name,cus_name_standard,cus_name_subid,ar_buyer_region,cus_region,ar_buyer_seg,cus_name_seg,cus_name_seg_str,ar_buyer_seg_str,match_score
430,000004.SZ,国华网安,福建乐游网络科技有限公司,福建乐游网络科技有限公司,29,福建乐游网络科技有限公司,福建乐游网络科技有限公司,11,福建,福建,"[福建, 乐游, 网络科技, 有限公司]","[福建, 乐游, 网络科技, 有限公司]",福建 乐游 网络科技 有限公司,福建 乐游 网络科技 有限公司,100
446,000004.SZ,国华网安,福建众邦药业有限公司,福建众邦药业有限公司,30,福建众邦药业有限公司,福建众邦药业有限公司,12,福建,福建,"[福建, 众邦, 药业, 有限公司]","[福建, 众邦, 药业, 有限公司]",福建 众邦 药业 有限公司,福建 众邦 药业 有限公司,100
462,000004.SZ,国华网安,福建摩格网络科技有限公司,福建摩格网络科技有限公司,31,福建摩格网络科技有限公司,福建摩格网络科技有限公司,13,福建,福建,"[福建, 摩格, 网络科技, 有限公司]","[福建, 摩格, 网络科技, 有限公司]",福建 摩格 网络科技 有限公司,福建 摩格 网络科技 有限公司,100
551,000006.SZ,深振业A,天虹商场股份有限公司,天虹商场股份有限公司,2,天虹商场股份有限公司,天虹商场股份有限公司,3,None,None,"[天虹, 商场, 股份, 有限公司]","[天虹, 商场, 股份, 有限公司]",天虹 商场 股份 有限公司,天虹 商场 股份 有限公司,100
599,000006.SZ,深振业A,深圳市前海同创汇产业运营管理有限公司,深圳市前海同创汇产业运营管理有限公司,7,深圳市前海同创汇产业运营管理有限公司,深圳市前海同创汇产业运营管理有限公司,6,深圳,深圳,"[深圳市, 前海同, 创汇, 产业, 运营, 管理, 有限公司]","[深圳市, 前海同, 创汇, 产业, 运营, 管理, 有限公司]",深圳市 前海同 创汇 产业 运营 管理 有限公司,深圳市 前海同 创汇 产业 运营 管理 有限公司,100
619,000006.SZ,深振业A,深圳市南山区建筑工务署,深圳市南山区建筑工务署,9,深圳市南山区建筑工务署,深圳市南山区建筑工务署,8,深圳,深圳,"[深圳市, 南山区, 建筑, 工务, 署]","[深圳市, 南山区, 建筑, 工务, 署]",深圳市 南山区 建筑 工务 署,深圳市 南山区 建筑 工务 署,100
797,000007.SZ,全新好,MAPLE PACKING INDUSTRIAL (H.K.) LIMITED,MAPLE PACKING INDUSTRIAL (H.K.) LIMITED,5,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,1,None,None,"[MAPLE, , PACKING, , INDUSTRIAL, , (, H, .,...","[MAPLEPACKINGINDUSTRIAL, (, H, ., K, ., ), LIM...",maplepackingindustrialhklimited,maplepackingindustrialhklimited,100
814,000007.SZ,全新好,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,6,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,1,None,None,"[MAPLEPACKINGINDUSTRIAL, (, H, ., K, ., ), LIM...","[MAPLEPACKINGINDUSTRIAL, (, H, ., K, ., ), LIM...",maplepackingindustrialhklimited,maplepackingindustrialhklimited,100
831,000007.SZ,全新好,MAPLEPACKINGINDUSTRIALLIMITED,MAPLEPACKINGINDUSTRIALLIMITED,7,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,MAPLEPACKINGINDUSTRIAL(H.K.)LIMITED,1,None,None,[MAPLEPACKINGINDUSTRIALLIMITED],"[MAPLEPACKINGINDUSTRIAL, (, H, ., K, ., ), LIM...",maplepackingindustrialhklimited,maplepackingindustriallimited,97
900,000007.SZ,全新好,WANDE INDUSTRIAL (HK) LIMITED,WANDE INDUSTRIAL (HK) LIMITED,11,WANDEINDUSTRIAL(HK)LIMITED,WANDEINDUSTRIAL(HK)LIMITED,2,None,None,"[WANDE, , INDUSTRIAL, , (, HK, ), , LIMITED]","[WANDEINDUSTRIAL, (, HK, ), LIMITED]",wandeindustrialhklimited,wandeindustrialhklimited,100


In [81]:
output_path3 = os.path.join(root_dir, "df_fuzzymatch90_v2.csv")
df_fuzzymatch90.to_csv(output_path3, encoding = 'utf-8-sig')

# Task 2: Companies classified (government/SOE/POE) 

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import re

In [10]:
# Keep the key columns and drop duplicate data
df1 = df.iloc[:,:4]
df1 = df1.drop_duplicates(subset = ['ar_buyer_standard'])

### Step 1: Crawling the SOE directory from the website

In [5]:
# Extract company names in one page.
def extract_company_names(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    try:
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8'  # Ensure proper encoding for Chinese content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Locate all containers that include company information
        company_blocks = soup.select('div.sop')  # Locate by class name
        
        companies = []
        for block in company_blocks:
            # Extract text and split with separater to avoid sticking 
            text = block.get_text(strip=True, separator='|') 
            if "企业名称：" in text:
                # irectly extract <span> with class 'dvright'
                name_span = block.find('span', class_='dvright')
                if name_span:
                    company_name = name_span.text.strip()
                    companies.append(company_name)
        
        return companies
    
    except Exception as e:
        print(f"Request failed: {e}")
        return []


In [7]:
BASE_URL = 'http://www.guozi.org/certification/directory_index.php'
HEADERS = {'User-Agent': 'Mozilla/5.0'}

def get_total_pages(url):
    """Get total pages"""
    try:
        response = requests.get(url, headers=HEADERS)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the element that contains total page information
        page_info = soup.find('div', class_='viciao').get_text(strip=True)
        print("Page info:", page_info)  # Print page_info
        
        # Use regex to extract the number of pages
        match = re.search(r'共\s*(\d+)\s*页', page_info)
        
        if match:
            return int(match.group(1))  # Return total pages
        else:
            print("未找到总页数信息")
            return 1  # Default to 1
    
    except Exception as e:
        print(f"Error: {e}")
        return 1  # Default to 1 if page not found
    
total_pages = get_total_pages(BASE_URL)
print(f"Total pages: {total_pages}")

Page info: 上一页123456789下一页共126页1258家
Total pages: 126


In [8]:
def crawl_all_pages():
    """Crawl data from all pages"""
    total_pages = get_total_pages(BASE_URL)
    print(f"Total pages: {total_pages}")
    
    all_companies = []
    for page in range(1, total_pages + 1):
        print(f"正在抓取第 {page} 页的数据...")
        url = f"{BASE_URL}?page={page}"  # Construct URL dynamically
        companies = extract_company_names(url)
        all_companies.extend(companies)  # Add company names from this page to the list
        print(f"第 {page} 页抓取完成。\n")
    
    # Return to comapny list
    return all_companies

# Crawl data
companies = crawl_all_pages()

Page info: 上一页123456789下一页共126页1258家
Total pages: 126
正在抓取第 1 页的数据...
第 1 页抓取完成。

正在抓取第 2 页的数据...
第 2 页抓取完成。

正在抓取第 3 页的数据...
第 3 页抓取完成。

正在抓取第 4 页的数据...
第 4 页抓取完成。

正在抓取第 5 页的数据...
第 5 页抓取完成。

正在抓取第 6 页的数据...
第 6 页抓取完成。

正在抓取第 7 页的数据...
第 7 页抓取完成。

正在抓取第 8 页的数据...
第 8 页抓取完成。

正在抓取第 9 页的数据...
第 9 页抓取完成。

正在抓取第 10 页的数据...
第 10 页抓取完成。

正在抓取第 11 页的数据...
第 11 页抓取完成。

正在抓取第 12 页的数据...
第 12 页抓取完成。

正在抓取第 13 页的数据...
第 13 页抓取完成。

正在抓取第 14 页的数据...
第 14 页抓取完成。

正在抓取第 15 页的数据...
第 15 页抓取完成。

正在抓取第 16 页的数据...
第 16 页抓取完成。

正在抓取第 17 页的数据...
第 17 页抓取完成。

正在抓取第 18 页的数据...
第 18 页抓取完成。

正在抓取第 19 页的数据...
第 19 页抓取完成。

正在抓取第 20 页的数据...
第 20 页抓取完成。

正在抓取第 21 页的数据...
第 21 页抓取完成。

正在抓取第 22 页的数据...
第 22 页抓取完成。

正在抓取第 23 页的数据...
第 23 页抓取完成。

正在抓取第 24 页的数据...
第 24 页抓取完成。

正在抓取第 25 页的数据...
第 25 页抓取完成。

正在抓取第 26 页的数据...
第 26 页抓取完成。

正在抓取第 27 页的数据...
第 27 页抓取完成。

正在抓取第 28 页的数据...
第 28 页抓取完成。

正在抓取第 29 页的数据...
第 29 页抓取完成。

正在抓取第 30 页的数据...
第 30 页抓取完成。

正在抓取第 31 页的数据...
第 31 页抓取完成。

正在抓取第 32 页的数据...
第 32 页抓取完成。

正在抓取

In [28]:
all_companies = pd.DataFrame(companies)
output_company = os.path.join(root_dir,"国企list.csv")
all_companies.to_csv(output_company, encoding = 'utf-8-sig')

### Construct a government department list

In [13]:
government_department = {"政府", "公安", "派出所", "财政局", "教育局",
                         "发改委", "税务局", "法院", "检察院", "交通局", 
                         "人力资源和社会保障局", "农业农村局", "林业局", 
                         "城市管理", "城管", "信访局", "外交部", "财政",
                         "自然资源局", "住房和城乡建设部", "交通运输部",
                         "商务部", "工业和信息化部", "中国人民银行",
                         "审计署", "国资委", "工务署"}

#### Use Jieba to seperate the string to words.

In [20]:
# Use jieba to seperate the words.
df_companymatch = df1.copy()
df_companymatch['ar_buyer_seg'] = df_companymatch['ar_buyer_standard'].astype(str).apply(lambda x: jieba.lcut(x))
df_companymatch['ar_buyer_seg_str'] = df_companymatch['ar_buyer_seg'].apply(lambda x: ' '.join(x))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\SHUNIN~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.336 seconds.
Prefix dict has been built successfully.


### Step 2: Remove common company suffixes, also extract the city name

In [15]:
# Remove common company suffixes
def clean_company_name(name):
    name = str(name)
    name = re.sub(r"(有限责任公司|股份有限公司|有限公司|公司)$", "", name) 
    return name.strip()

In [21]:
# Extract the city name for row data
df_companymatch['ar_region'] = df_companymatch['ar_buyer_standard'].astype(str).apply(extract_region)

In [28]:
# Extract the city name for SOE companies
SOE_company_list = pd.DataFrame({'name': companies})
SOE_company_list['region'] = SOE_company_list['name'].astype(str).apply(extract_region)

### Step 3: Fuzzy match

In [23]:
# Match the government departments
def fuzzy_match_government(text, keywords):
    text_clean = clean_company_name(text)
    best_score = 0
    best_keyword = None
    for keyword in keywords:
        keyword_clean = clean_company_name(keyword)
        score = fuzz.partial_ratio(keyword_clean , text_clean)
        if score > best_score:
            best_score = score
            best_keyword = keyword
    return pd.Series([best_keyword, best_score])

# 应用到 DataFrame
df_companymatch[['government_matched_keyword', 'government_match_score']] = df_companymatch['ar_buyer_standard'].apply(
    lambda x: fuzzy_match_government(x, government_department)
)

In [35]:
# Match the SOE company list
def fuzzy_match_company(text, region, keywords_df):
    text_clean = clean_company_name(text)
    best_score = 0
    best_keyword = None
    
    # Delete companies with different regions.
    region_keywords = keywords_df[(keywords_df['region'] == region) | (keywords_df['region'].isnull())]
    
    for keyword in region_keywords['name']:
        keyword_clean = clean_company_name(keyword)
        #score = fuzz.token_sort_ratio(text_clean, keyword_clean) 
        score = fuzz.token_set_ratio(text_clean, keyword_clean) 
        if score > best_score:
            best_score = score
            best_keyword = keyword
    return pd.Series([best_keyword, best_score])

df_companymatch[['company_matched_keyword', 'company_match_score']] = df_companymatch.apply(
    lambda row: fuzzy_match_company(row['ar_buyer_standard'], row['ar_region'], SOE_company_list),
    axis = 1
)

In [44]:
import numpy as np

conditions = [
    df_companymatch['government_match_score'] >= 60,
    df_companymatch['company_match_score'] >= 50
]

choices = [1, 2]

df_companymatch['company_type'] = np.select(conditions, choices, default=3)

In [47]:
outpath4 = os.path.join(root_dir,"ar_company_type.csv")
df_companymatch.to_csv(outpath4, encoding = 'utf-8-sig')